# Running Chains on Traced Datasets

Developing applications with language models can be uniquely challenging. To manage this complexity and ensure reliable performance, LangChain provides tracing and evaluation functionality through . This notebook demonstrates how to run Chains, which are language model functions, on previously captured datasets or traces. Some common use cases for this approach include:

- Running an evaluation chain to grade previous runs.
- Comparing different chains, LLMs, and agents on traced datasets.
- Executing a stochastic chain multiple times over a dataset to generate metrics before deployment.

Please note that this notebook assumes you have LangChain+ tracing running in the background. To set it up, follow the [tracing directions here](..\/..\/tracing\/local_installation.md).


In [2]:
from langchain.client import LangChainPlusClient

client = LangChainPlusClient(
    api_url="http://localhost:8000",
    api_key=None,
    #tenant_id="", Required for connecting to a hosted tenant
)

In [5]:
client.tenant_id

'309d7a02-99cc-4a5f-abf0-2b389747bb95'

In [6]:
from langchain import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [26]:
llm = OpenAI()
llm.generate(["Hello, world!"], callbacks=[tracer])

LLMResult(generations=[[Generation(text="\n\nMy name is Adam and I'm excited to be part of this community. I'm a big fan of technology, and I'm always looking for ways to learn and grow. I'm also passionate about helping others, and I'm always looking for new opportunities to do so. I look forward to connecting with all of you!", generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 4, 'total_tokens': 71}, 'model_name': 'text-davinci-003'})

In [29]:
from langchain.callbacks.tracers.langchain import LangChainTracerV2
tracer = LangChainTracerV2(session_name='default')
tracer.load_session("default")
model = ChatOpenAI() # callbacks=[tracer])
messages = [[
    HumanMessage(content="I am human roar"),
    AIMessage(content="I am AI beep boop"),
    SystemMessage(content="I am a system message"),
    ]
]
results = model.generate(messages=messages, callbacks=[tracer])

## Seed an example dataset

If you have been using LangChainPlus already, you may have datasets available. To view all saved datasets, run:

```
datasets = client.list_datasets()
datasets
```
Datasets can be created in a number of ways, most often by collecting `Run`'s captured through the LangChain tracing API.

However, this notebook assumes you're running locally for the first time, so we'll start by uploading an example evaluation dataset.

In [ ]:
# !pip install datasets > /dev/null
# !pip install pandas > /dev/null

In [ ]:
import pandas as pd
from langchain.evaluation.loading import load_dataset

dataset = load_dataset("agent-search-calculator")
df = pd.DataFrame(dataset, columns=["question", "answer"])
df.columns = ["input", "output"] # The chain we want to evaluate below expects inputs with the "input" key 

In [ ]:
df.head()

In [ ]:
dataset_name = f"calculator_example.csv"

In [ ]:
if dataset_name not in set([dataset.name for dataset in client.list_datasets()]):
    dataset = client.upload_dataframe(df, 
                            name=dataset_name,
                            description="Acalculator example dataset",
                            input_keys=["input"],
                            output_keys=["output"],
                   )

## Running a Chain on a Traced Dataset

Once you have a dataset, you can run a chain over it to see its results. The run traces will automatically be associated with the dataset for easy attribution and analysis.

**First, we'll define the chain we wish to run over the dataset.**

In this case, we're using an agent, but it can be any simple chain.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools
from langchain.agents import AgentType

llm = ChatOpenAI(temperature=0)
tools = load_tools(['serpapi', 'llm-math'], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False)

**Now we're ready to run the chain!**

In [ ]:
chain_results = await client.arun_on_dataset(
    dataset_name=dataset_name,
    chain=agent,
    num_workers=5, # Optional, sets the number of examples to run at a time
    session_name="Calculator Dataset Runs", # Optional. Will be sent to the 'default' session otherwise
)

## Reviewing the Chain Results

The method called above returns a dictionary mapping Example IDs to the output of the chain.
You can directly inspect the results below.

In [ ]:
chain_results

In [ ]:
# You can navigate to the UI by clicking on the link below
client

In [ ]:
# You can review all the chain runs over a given example as follows:
example_id = next(iter(chain_results))
example = client.read_example(example_id)

In [ ]:
# For example, view the chain runs on this example
example.chain_runs